# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st

from numpy.random import RandomState

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from tqdm.notebook import tqdm


SEED = 42

## Загрузка и подготовка данных

In [2]:
try:
    df1 = pd.read_csv('../datasets/geo_data_0.csv')
    df2 = pd.read_csv('../datasets/geo_data_1.csv')
    df3 = pd.read_csv('../datasets/geo_data_2.csv')
except: 
    df1 = pd.read_csv('practicum/module_2/datasets/geo_data_0.csv')
    df2 = pd.read_csv('practicum/module_2/datasets/geo_data_1.csv')
    df3 = pd.read_csv('practicum/module_2/datasets/geo_data_2.csv')

df1.shape, df2.shape, df3.shape

((100000, 5), (100000, 5), (100000, 5))

In [3]:
# Посмотрим на дубликаты в столбце id 
df1['id'].duplicated().sum(), df2['id'].duplicated().sum(), df3['id'].duplicated().sum()

(10, 4, 4)

**Видим, что дубликаты есть, но их немного. Можно лишь предположить, что это уточненные данные по скважине. Поскольку установить какие данные более актуальные невозможно, оставим наблюдения, на результаты модели сильно повлиять они не должны. Столбец id из датасета удалим**

In [4]:
# Разделение на train и target
X_1 = df1.drop(['product', 'id'], axis=1)
X_2 = df2.drop(['product', 'id'], axis=1)
X_3 = df3.drop(['product', 'id'], axis=1)

y_1 = df1['product']
y_2 = df2['product']
y_3 = df3['product']

In [5]:
# Разделение на train и valid

X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(X_1, y_1,
                                                             test_size=.25,
                                                             random_state=SEED,
                                                             )

X_2_train, X_2_valid, y_2_train, y_2_valid = train_test_split(X_2, y_2,
                                                             test_size=.25,
                                                             random_state=SEED
                                                             )

X_3_train, X_3_valid, y_3_train, y_3_valid = train_test_split(X_3, y_3,
                                                             test_size=.25,
                                                             random_state=SEED
                                                             )

## Обучение и проверка модели

### Первый регион

In [53]:
def true_pred(model,X_train, y_train, X_valid, y_valid, verbose:bool = False, name:str=None):
    model.fit(X_train, y_train)
    y_hat = model.predict(X_valid)
    true_pred = pd.DataFrame([y_valid.reset_index(drop=True), y_hat]).T
    true_pred.columns = ['true', 'predicted']
    
    mean = true_pred['predicted'].mean()
    rmse = mean_squared_error(y_valid, y_hat, squared=False)
    if verbose:
        print('-'*80)
        print(name)
        print(f"Среднее по региону: {mean:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print('-'*80)
    
    return true_pred

In [56]:
lr_model = LinearRegression()

true_pred_1 = true_pred(lr_model, 
                        X_1_train, 
                        y_1_train, 
                        X_1_valid, 
                        y_1_valid, 
                        verbose=True, 
                        name='Регион 1'
                       )

true_pred_2 = true_pred(lr_model, 
                        X_2_train, 
                        y_2_train, 
                        X_2_valid, 
                        y_2_valid, 
                        verbose=True, 
                        name='Регион 2'
                       )

true_pred_3 = true_pred(lr_model, 
                        X_3_train,
                        y_3_train,
                        X_3_valid, 
                        y_3_valid, 
                        verbose=True, 
                        name='Регион 3'
                       )

--------------------------------------------------------------------------------
Регион 1
Среднее по региону: 92.40
RMSE: 37.76
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Регион 2
Среднее по региону: 68.71
RMSE: 0.89
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Регион 3
Среднее по региону: 94.77
RMSE: 40.15
--------------------------------------------------------------------------------


### Вывод:
* Лучше всего себя показала вторая модель. Минимальный уровень ошибки из трех.
* Первая и тетья модели имеют схожие показатели, как по среднему запасу так и по средней ошибке.

## Подготовка к расчёту прибыли

In [31]:
REVENUE_PER_BARREL = 450_000
INVEST = 10_000_000_000
SAMPLE_SIZE = 500
SUBSAMPLE_SIZE = 200

In [33]:
# Достаточный объём сырья для безубыточной разработки новой скважины:

profit_threshold = INVEST / REVENUE_PER_BARREL / SUBSAMPLE_SIZE
print(f"Для безубыточности необходимо иметь запасы не менее {round(profit_threshold,2)} тыс. баррелей ")

compare = pd.DataFrame(data=[profit_threshold,
                             true_pred_1['predicted'].mean(), 
                             true_pred_2['predicted'].mean(), 
                             true_pred_3['predicted'].mean(), 
                             ])
compare.columns = ['запасы тыс. баррелей']
compare.index = ['Порог безубыточности', 'Первый регион', 'Второй регион', 'Третий регион']
compare

Для безубыточности необходимо иметь запасы не менее 111.11 тыс. баррелей 


,запасы тыс. баррелей
Порог безубыточности,111.111111
Первый регион,92.398800
Второй регион,166.582271
Третий регион,93.861233


### Выводы:
* Средние запасы по всем трем регионам не преодалевают порог безубыточности. 

## Расчёт прибыли и рисков 

In [57]:
def profit_calc(pred_true: pd.DataFrame, size: int, price:int = REVENUE_PER_BARREL, invest=INVEST):
    pred_true = pred_true.sort_values(by='predicted', ascending=False)[:size] # Сортируем и оставляем только первые n значений
    total_reserves = pred_true['true'].sum() # Сумируем целевой признак
    region_revenue = total_reserves * price # умножаем на стоимость тысячи баррелей
    return region_revenue - INVEST

In [58]:
region_revenue = pd.DataFrame(
    [profit_calc(true_pred_1, SUBSAMPLE_SIZE),
    profit_calc(true_pred_2, SUBSAMPLE_SIZE),
    profit_calc(true_pred_3, SUBSAMPLE_SIZE)])

region_revenue.index = ['Первый регион', 'Второй регион', 'Третий регион']
region_revenue /= 1e9
region_revenue = region_revenue.round(3)
region_revenue.columns = ['млрд. руб.']
region_revenue

,млрд. руб.
Первый регион,3.359
Второй регион,2.415
Третий регион,2.599


In [72]:
def bootstrap_revenue(sbs, sample_size, subsample_size, state=RandomState(12345), bootstrap_iter:int = 1000, alpha:float = .05, 
                      name:str = None):
    bootstrap_profit = []     # Список для хранения значений
    loss_count = 0
    for i in tqdm(range(bootstrap_iter)):
        sample = sbs.sample(SAMPLE_SIZE, replace=True, random_state=state) # делаем выборку
        profit = profit_calc(sample, SUBSAMPLE_SIZE)                          # Считаем прибыль
        bootstrap_profit.append(profit)                             # Добавляем в список          
        if profit < 0: 
            loss_count += 1
    bootstrap_profit = pd.Series(bootstrap_profit) / 1e9            # Делим значения на 1млрд. для отображения в млрд. руб.
    bootstrap_profit = bootstrap_profit.round(3)
    print('-'*80)
    if name: 
        print(name)    
    print(f"Средняя прибыль: {bootstrap_profit.mean():.2f} млрд. руб.")
    print(f"{1 - alpha:.2%} доверительный интервал: \
                {st.t.interval(alpha, bootstrap_iter - 1, bootstrap_profit.mean(), bootstrap_profit.sem())}")
    print(f"Риск убытка {loss_count / bootstrap_iter:.2%}")
    print('-'*80)

In [73]:
# Регион 1
bootstrap_revenue(true_pred_1, SAMPLE_SIZE, SUBSAMPLE_SIZE, name='Регион 1')
# Регион 2
bootstrap_revenue(true_pred_2, SAMPLE_SIZE, SUBSAMPLE_SIZE, name='Регион 2')
# Регион 3
bootstrap_revenue(true_pred_3, SAMPLE_SIZE, SUBSAMPLE_SIZE, name='Регион 3')

  0%|          | 0/1000 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
Регион 1
Средняя прибыль: 0.41 млрд. руб.
95.00% доверительный интервал:                 (0.4057569647337225, 0.40679903526627753)
Риск убытка 6.70%
--------------------------------------------------------------------------------


  0%|          | 0/1000 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
Регион 2
Средняя прибыль: 0.44 млрд. руб.
95.00% доверительный интервал:                 (0.4411060577775202, 0.4419139422224798)
Риск убытка 1.60%
--------------------------------------------------------------------------------


  0%|          | 0/1000 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
Регион 3
Средняя прибыль: 0.39 млрд. руб.
95.00% доверительный интервал:                 (0.3846712878033612, 0.3857527121966388)
Риск убытка 7.80%
--------------------------------------------------------------------------------


### Итоговые выводы:

* Чтобы разработка была безубыточна, необходимо иметь сумарный запас не менее 111 тысяч баррелей в регионе.
* По итогам исследования все три региона преодалевают планку безубыточности но риски убытка различны.
* Доверительный интервал довольно узкий, что говорит о высокой надёжности результатов.
* По итогам исследования второй регион обладает наибольшим потенциалом, т.к. наиболее прибыльный, а так же имеет наименьший риск убытка. 


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован